In [94]:
import pandas as pd
import csv
import os
import statistics
from tqdm import tqdm

In [95]:
def check_max_significance(lst):
    max_value = max(lst)
    sorted_values = sorted(lst, reverse=True)
    second_largest_value = sorted_values[1]

    if max_value > 1.1 * second_largest_value and sum(counts) > 2:
        return lst.index(max_value) + 1
    else:
        return 404

def cal_score(gene):
    score = rankings[gene].max() / rankings[gene][i]
    return score

def cal_mean_score(cluster):
    cluster_score = []
    for key in cluster:
        cluster_score.append(gene_scores[key])
    if len(cluster_score) > 1:
        mean_score = sum(cluster_score) / len(cluster_score)
        # score = min(cluster_score) * 0.4 + mean_score * 0.6
        score = mean_score * 0.7 + min(cluster_score) * 0.3
    else:
        score = cluster_score[0]
    return score


def get_result(result):
    mapping = {1: 'B_cells', 2: 'T_cells', 3: 'NK_cells',
               4:'Neutrophils', 5: 'Macrophages ', 6: 'Fibroblasts ', 
               7: 'Endothelial_cells'}
    return mapping.get(result, 'Undefined')

In [96]:
##chicken
B_cells = ['Dennd5b','Pax5']
T_cells = ['Cd28']
NK_cells = ['Il2rb','Cd7']
Neutrophils = ['G0s2']
Macrophages = ['Il1r2']
Fibroblasts = ['Col1a1','Apoa1']
#Epithelial_cells = ['Sftpc']
Endothelial_cells = ['Npr3','Vwf']

In [97]:
column_names = ['Dennd5b', 'Pax5', 'Cd28', 'Il2rb',
                'Cd7', 'Il1r2', 'Col1a1', 'Apoa1', 'G0s2',
                'Npr3', 'Vwf']

In [98]:
cluster_lst = [B_cells, T_cells, NK_cells, Macrophages, Neutrophils,Fibroblasts, Endothelial_cells]

In [63]:
sample = 'CK'
samples = [file for file in os.listdir('./matrix') if file.startswith(sample)]
print(samples)


for file in samples:
    filename = f'./matrix/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    #os.makedirs(f'./matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'./matrix_cluster/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

['CK-I-20X.csv', 'CK-I-63X.csv', 'CK-M-20X.csv', 'CK-M-63X.csv']


Processing files: 100%|██████████| 75886/75886 [01:18<00:00, 970.31it/s] 


cluster
Undefined            26779
T_cells              21696
Fibroblasts          13851
NK_cells              6797
B_cells               4598
Macrophages           1015
Epithelial_cells      1001
Endothelial_cells      149
Name: count, dtype: int64


Processing files: 100%|██████████| 10293/10293 [00:04<00:00, 2331.09it/s]


cluster
T_cells              4322
Undefined            3662
Fibroblasts           831
Epithelial_cells      486
B_cells               453
NK_cells              381
Macrophages           126
Endothelial_cells      32
Name: count, dtype: int64


Processing files: 100%|██████████| 27400/27400 [00:15<00:00, 1737.18it/s]


cluster
Fibroblasts          6967
Undefined            6535
NK_cells             6504
T_cells              3275
B_cells              2465
Macrophages           745
Epithelial_cells      512
Endothelial_cells     397
Name: count, dtype: int64


Processing files: 100%|██████████| 6735/6735 [00:02<00:00, 2490.69it/s]


cluster
Undefined            2028
T_cells              1568
Fibroblasts          1147
B_cells               725
Epithelial_cells      527
Macrophages           327
NK_cells              320
Endothelial_cells      93
Name: count, dtype: int64


In [99]:
from tqdm import tqdm

In [107]:
sample = 'CK'
samples = os.listdir(f'./matrix/{sample}')
output = 'Epithelial_cells'
os.makedirs(output, exist_ok=True)
os.makedirs('Others', exist_ok=True)

for file in samples:
    print(file)
    filename = f'./matrix/{sample}/{file}'
    df = pd.read_csv(filename)
    filtered_df = df[df['Sftpc'] > 4]
    filtered_df['cluster'] = 'Epithelial_cells'
    cluster_counts = filtered_df['cluster'].value_counts()
    print(cluster_counts)
    filtered_df.to_csv(f'./{output}/{file}', index=False)
    Others = df[df['Sftpc'] < 5]
    Others.to_csv(f'./Others/{file}', index=False)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './matrix/CK'

In [103]:
import os

sample = 'CK'
absolute_path = os.path.abspath(f'./matrix/{sample}')
samples = os.listdir(absolute_path)
output = 'Epithelial_cells'
os.makedirs(output, exist_ok=True)


FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'e:\\manuscript\\单细胞流感跨物种\\细胞类型的定义\\20240611\\matrix\\CK'

In [41]:
sample = 'CK'
samples = os.listdir(f'./Others/{sample}')

for file in samples:
    filename = f'./Others/{sample}/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    os.makedirs(f'matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'matrix_cluster/{sample}/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './Others/CK'

In [40]:
pwd

'e:\\manuscript\\单细胞流感跨物种\\细胞类型的定义\\20240611'

In [53]:
sample = 'CK'
samples = os.listdir(f'./Others/CK')

for file in samples:
    df1 = pd.read_csv(f'./matrix_cluster/CK/{file}')
    df2 = pd.read_csv(f'./Neutrophils/CK/{file}')
    print(file)
    # 使用pd.concat()函数将两个DataFrame合并
    merged_df = pd.concat([df1, df2], ignore_index=True)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)
    # 保存合并后的DataFrame为CSV文件
    merged_df.to_csv(f'./CK/{file}', index=False)


CK-I-20X.csv
cluster
Undefined            37419
T_cells              18036
NK_cells              4528
B_cells               1981
Macrophages           1870
Fibroblasts           1808
Endothelial_cells     1240
Epithelial_cells       929
Neutrophils             91
Name: count, dtype: int64
CK-I-63X.csv
cluster
Undefined            3885
T_cells              2854
NK_cells             1433
Neutrophils           660
Fibroblasts           634
B_cells               511
Endothelial_cells     350
Macrophages           210
Epithelial_cells       91
Name: count, dtype: int64
CK-M-20X.csv
cluster
Undefined            9210
T_cells              6589
Endothelial_cells    2369
NK_cells             2295
Fibroblasts          1775
B_cells              1713
Macrophages          1115
Epithelial_cells      903
Neutrophils           225
Name: count, dtype: int64
CK-M-63X.csv
cluster
T_cells              1624
Neutrophils          1323
Endothelial_cells    1302
Undefined            1010
NK_cells              5

In [16]:
def check_max_significance(lst):
    max_value = max(lst)
    sorted_values = sorted(lst, reverse=True)
    second_largest_value = sorted_values[1]

    if max_value > 1.1 * second_largest_value and sum(counts) > 2:
        return lst.index(max_value) + 1
    else:
        return 404

def cal_score(gene):
    score = rankings[gene].max() / rankings[gene][i]
    return score

def cal_mean_score(cluster):
    cluster_score = []
    for key in cluster:
        cluster_score.append(gene_scores[key])
    if len(cluster_score) > 1:
        mean_score = sum(cluster_score) / len(cluster_score)
        # score = min(cluster_score) * 0.4 + mean_score * 0.6
        score = mean_score * 0.7 + min(cluster_score) * 0.3
    else:
        score = cluster_score[0]
    return score

In [19]:
##pig
B_cells = ['Cd79b','Pax5']
T_cells = ['Il7r']
NK_cells = ['Klrk1',"Il12rb2"]
Neutrophils = ['Ifit2','Itga2']
Macrophages = ['C1qa','C1qb','C1qc']
Fibroblasts = ['Col1a2','Dcn','Pi16']
Epithelial_cells = ['Samd12','Rtkn2']
Endothelial_cells = ['Slpi']


column_names = ['Cd79b','Pax5','Il7r','Klrk1','Ifit2','Itga2','C1qa','C1qb','C1qc','Col1a2','Dcn','Pi16','Samd12','Rtkn2','Slpi',"Il12rb2"]


def get_result(result):
    mapping = {1: 'B_cells', 2: 'T_cells', 3: 'NK_cells', 4: 'Neutrophils',
               5:'Macrophages', 6: 'Fibroblasts ', 7: 'Epithelial_cells',
               8: 'Endothelial_cells'}
    return mapping.get(result, 'Undefined')

cluster_lst = [B_cells, T_cells, NK_cells, Neutrophils, Macrophages, Fibroblasts, Epithelial_cells, Endothelial_cells]

In [20]:

sample = 'PIG'
samples = [file for file in os.listdir('./matrix') if file.startswith(sample)]
print(samples)


for file in samples:
    filename = f'./matrix/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    #os.makedirs(f'./matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'./matrix_cluster/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

['PIG-I-20X.csv', 'PIG-I-63X.csv', 'PIG-M-20X.csv', 'PIG-M-63X.csv']


Processing files: 100%|██████████| 33277/33277 [00:27<00:00, 1224.30it/s]


cluster
Undefined            14747
Macrophages           6565
B_cells               3847
Fibroblasts           2420
T_cells               2285
Endothelial_cells     1631
Neutrophils           1133
Epithelial_cells       621
NK_cells                28
Name: count, dtype: int64


Processing files: 100%|██████████| 8943/8943 [00:04<00:00, 1919.89it/s]


cluster
Undefined            2287
Fibroblasts          1776
B_cells              1409
Macrophages          1231
Neutrophils           861
Epithelial_cells      508
T_cells               476
Endothelial_cells     353
NK_cells               42
Name: count, dtype: int64


Processing files: 100%|██████████| 14778/14778 [00:08<00:00, 1676.69it/s]


cluster
Macrophages          2867
B_cells              2767
Endothelial_cells    2481
T_cells              2047
Undefined            1745
Fibroblasts          1428
Epithelial_cells      869
Neutrophils           546
NK_cells               28
Name: count, dtype: int64


Processing files: 100%|██████████| 4121/4121 [00:01<00:00, 2156.82it/s]

cluster
B_cells              765
Macrophages          661
Undefined            602
Endothelial_cells    597
Fibroblasts          555
T_cells              360
Neutrophils          280
Epithelial_cells     271
NK_cells              30
Name: count, dtype: int64


In [13]:
def check_max_significance(lst):
    max_value = max(lst)
    sorted_values = sorted(lst, reverse=True)
    second_largest_value = sorted_values[1]

    if max_value > 1.1 * second_largest_value and sum(counts) > 2:
        return lst.index(max_value) + 1
    else:
        return 404

def cal_score(gene):
    score = rankings[gene].max() / rankings[gene][i]
    return score

def cal_mean_score(cluster):
    cluster_score = []
    for key in cluster:
        cluster_score.append(gene_scores[key])
    if len(cluster_score) > 1:
        mean_score = sum(cluster_score) / len(cluster_score)
        score = max(cluster_score) * 0.4 + mean_score * 0.6
        # score = mean_score * 0.6 + min(cluster_score) * 0.4
    else:
        score = cluster_score[0]
    return score

In [14]:
##MS
# ,'Agbl1'

B_cells = ['Cd79a','Bank1']
T_cells = ['St8sia6']
NK_cells = ['Ccl5','Gzma']
Neutrophils = ['S100a8','S100a9','Retnlg']
Macrophages = ['Kcnip4','Chil3']
Fibroblasts = ['Col3a1','Col1a2']
Epithelial_cells = ['Sftpb','Sftpa1']
Endothelial_cells = ['Prickle2','Cldn5']


column_names = ['Cd79a','Bank1','St8sia6','Ccl5','Gzma',
                'S100a8','S100a9','Retnlg','Kcnip4','Chil3',
                'Col3a1','Col1a2','Sftpb','Sftpa1',
                'Prickle2','Cldn5']


def get_result(result):
    mapping = {1: 'B_cells', 2: 'T_cells', 3: 'NK_cells', 4: 'Neutrophils',
               5:'Macrophages', 6: 'Fibroblasts ', 7: 'Epithelial_cells',
               8: 'Endothelial_cells'}
    return mapping.get(result, 'Undefined')

cluster_lst = [B_cells, T_cells, NK_cells, Macrophages, Fibroblasts, Epithelial_cells, Endothelial_cells, Neutrophils]

In [15]:
sample = 'MS'
samples = [file for file in os.listdir('./matrix') if file.startswith(sample)]
print(samples)


for file in samples:
    filename = f'./matrix/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    #os.makedirs(f'./matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'./matrix_cluster/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

['MS-I-20X.csv', 'MS-I-63X.csv', 'MS-M-20X.csv', 'MS-M-63X.csv']


Processing files: 100%|██████████| 18542/18542 [00:12<00:00, 1544.74it/s]


cluster
Fibroblasts          5665
B_cells              3327
Macrophages          2824
Undefined            2672
Endothelial_cells    1229
NK_cells              856
Epithelial_cells      837
Neutrophils           807
T_cells               325
Name: count, dtype: int64


Processing files: 100%|██████████| 5361/5361 [00:02<00:00, 2054.29it/s]


cluster
Fibroblasts          1517
Undefined            1273
Epithelial_cells      582
Endothelial_cells     538
B_cells               418
Macrophages           408
Neutrophils           249
NK_cells              202
T_cells               174
Name: count, dtype: int64


Processing files: 100%|██████████| 22123/22123 [00:15<00:00, 1461.03it/s]


cluster
B_cells              5043
Fibroblasts          4471
Undefined            3695
Macrophages          2978
Endothelial_cells    2088
Epithelial_cells     1329
NK_cells             1273
T_cells               801
Neutrophils           445
Name: count, dtype: int64


Processing files: 100%|██████████| 3575/3575 [00:01<00:00, 2212.96it/s]

cluster
Fibroblasts          758
Undefined            585
Endothelial_cells    539
B_cells              495
Epithelial_cells     433
Macrophages          303
NK_cells             199
Neutrophils          147
T_cells              116
Name: count, dtype: int64


In [30]:
def check_max_significance(lst):
    max_value = max(lst)
    sorted_values = sorted(lst, reverse=True)
    second_largest_value = sorted_values[1]

    if max_value > 1.1 * second_largest_value and sum(counts) > 2:
        return lst.index(max_value) + 1
    else:
        return 404

def cal_score(gene):
    score = rankings[gene].max() / rankings[gene][i]
    return score

def cal_mean_score(cluster):
    cluster_score = []
    for key in cluster:
        cluster_score.append(gene_scores[key])
    if len(cluster_score) > 1:
        mean_score = sum(cluster_score) / len(cluster_score)
        # score = min(cluster_score) * 0.4 + mean_score * 0.6
        score = mean_score * 0.7 + min(cluster_score) * 0.3
    else:
        score = cluster_score[0]
    return score

In [31]:
##PG
#'Dscam','Hvcn1'
B_cells = ['Dscam','Hvcn1']
T_cells = ['Tcf7']
NK_cells = ['Pde1c']
Neutrophils = ['G0s2','Steap4']
Macrophages = ['Efemp1']
Fibroblasts = ['Thbs1','Dcn','Ism1']
Epithelial_cells = ['Fabp3']
Endothelial_cells = ['Prox1','Ebf3']

column_names = ['Dscam','Hvcn1','Tcf7','Pde1c',
                'G0s2','Steap4','Efemp1','Thbs1','Dcn','Ism1',
                'Fabp3','Prox1','Ebf3']


def get_result(result):
    mapping = {1: 'B_cells', 2: 'T_cells', 3: 'NK_cells',
               4:'Macrophages', 5: 'Fibroblasts ', 6: 'Epithelial_cells',
               7: 'Endothelial_cells', 8: 'Neutrophils'}
    return mapping.get(result, 'Undefined')

cluster_lst = [B_cells, T_cells, NK_cells, Macrophages, Fibroblasts, Epithelial_cells, Endothelial_cells, Neutrophils]

In [32]:
sample = 'PG'
samples = [file for file in os.listdir('./matrix') if file.startswith(sample)]
print(samples)


for file in samples:
    filename = f'./matrix/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    #os.makedirs(f'./matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'./matrix_cluster/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

['PG-I-20X.csv', 'PG-I-63X.csv', 'PG-M-20X.csv', 'PG-M-63X.csv']


Processing files: 100%|██████████| 26602/26602 [00:17<00:00, 1545.72it/s]


cluster
Undefined            9505
B_cells              3366
Neutrophils          2846
Macrophages          2737
Endothelial_cells    2350
T_cells              2333
Fibroblasts          1354
Epithelial_cells     1064
NK_cells             1047
Name: count, dtype: int64


Processing files: 100%|██████████| 8860/8860 [00:06<00:00, 1285.86it/s]


cluster
Undefined            5135
Neutrophils          1158
Macrophages           841
NK_cells              520
Fibroblasts           463
B_cells               307
T_cells               276
Epithelial_cells      150
Endothelial_cells      10
Name: count, dtype: int64


Processing files: 100%|██████████| 29528/29528 [00:21<00:00, 1360.11it/s]


cluster
Undefined            9877
B_cells              4638
Neutrophils          3798
Endothelial_cells    2764
Fibroblasts          2656
T_cells              2515
Macrophages          1810
NK_cells              758
Epithelial_cells      712
Name: count, dtype: int64


Processing files: 100%|██████████| 7633/7633 [00:03<00:00, 2257.33it/s]

cluster
Undefined            1893
Macrophages          1778
Neutrophils          1682
T_cells               592
B_cells               500
NK_cells              486
Fibroblasts           369
Epithelial_cells      191
Endothelial_cells     142
Name: count, dtype: int64


In [90]:
def check_max_significance(lst):
    max_value = max(lst)
    sorted_values = sorted(lst, reverse=True)
    second_largest_value = sorted_values[1]

    if max_value > 1.1 * second_largest_value and sum(counts) > 2:
        return lst.index(max_value) + 1
    else:
        return 404

def cal_score(gene):
    score = rankings[gene].max() / rankings[gene][i]
    return score

def cal_mean_score(cluster):
    cluster_score = []
    for key in cluster:
        cluster_score.append(gene_scores[key])
    if len(cluster_score) > 1:
        mean_score = sum(cluster_score) / len(cluster_score)
        # score = min(cluster_score) * 0.4 + mean_score * 0.6
        score = mean_score * 0.6 + min(cluster_score) * 0.4
    else:
        score = cluster_score[0]
    return score

In [91]:
##DOG
#'Dscam','Hvcn1'
B_cells = ['Cecr2']
T_cells = ['Il7r']
NK_cells = ['Il12rb2']
Neutrophils = ['Slc7a11','Cd4']
Macrophages = ['C1qb','Slc16a10']
Fibroblasts = ['Dcn','Col1a2']
Epithelial_cells = ['Sftpb']
Endothelial_cells = ['Cldn5','Cavin2']

column_names = ['Cecr2','Il7r','Il12rb2','Slc7a11','Cd4','C1qb','Slc16a10','Dcn','Col1a2','Sftpb','Cldn5','Cavin2']


def get_result(result):
    mapping = {1: 'B_cells', 2: 'T_cells', 3: 'NK_cells',
               4:'Macrophages', 5: 'Fibroblasts ', 6: 'Epithelial_cells',
               7: 'Endothelial_cells', 8: 'Neutrophils'}
    return mapping.get(result, 'Undefined')

cluster_lst = [B_cells, T_cells, NK_cells, Macrophages, Fibroblasts, Epithelial_cells, Endothelial_cells, Neutrophils]

In [93]:
sample = 'DOG'
samples = [file for file in os.listdir('./matrix') if file.startswith(sample)]
print(samples)


for file in samples:
    filename = f'./matrix/{file}'
    df = pd.read_csv(filename, index_col='cell')
    rankings = df.rank(method='min', ascending=False)
    with open('matrix-cluster-tmp.csv', mode='w', newline='') as output_file:
        writer = csv.writer(output_file)
        writer.writerow(['cluster'])  # 写出表头
        for i in tqdm(range(len(rankings)), desc='Processing files'):
            gene_scores = {}
            for gene in column_names:
                gene_score = cal_score(gene)
                gene_scores[gene] = gene_score
            cluster_scores = []
            for cluster in cluster_lst:
                cluster_score = cal_mean_score(cluster)
                cluster_scores.append(cluster_score)

            counts = df.iloc[i, :]
            result = check_max_significance(cluster_scores)
            output = get_result(result)
            row = [output]
            writer.writerow(row)

    df1 = pd.read_csv(filename, index_col=None)
    df2 = pd.read_csv('matrix-cluster-tmp.csv', index_col=None)
    merged_df = pd.concat([df1, df2], axis=1)

    #os.makedirs(f'./matrix_cluster/{sample}', exist_ok=True)
    id = file.split('.')[0]
    merged_df.to_csv(f'./matrix_cluster/{id}-cluster.csv', index=None)
    cluster_counts = merged_df['cluster'].value_counts()
    print(cluster_counts)

['DOG-I-20X.csv', 'DOG-I-63X.csv', 'DOG-M-20X.csv', 'DOG-M-63X.csv']


Processing files: 100%|██████████| 33692/33692 [00:21<00:00, 1551.87it/s]


cluster
Undefined            7021
Epithelial_cells     6248
B_cells              4819
Macrophages          4532
Endothelial_cells    3978
Fibroblasts          3890
Neutrophils          1953
NK_cells              748
T_cells               503
Name: count, dtype: int64


Processing files: 100%|██████████| 6578/6578 [00:02<00:00, 2530.74it/s]


cluster
Fibroblasts          1601
Undefined            1508
B_cells               822
Macrophages           779
Endothelial_cells     720
Epithelial_cells      637
Neutrophils           338
NK_cells              108
T_cells                65
Name: count, dtype: int64


Processing files: 100%|██████████| 25543/25543 [00:14<00:00, 1767.19it/s]


cluster
Fibroblasts          5890
Epithelial_cells     5374
Undefined            3765
Endothelial_cells    3598
B_cells              2469
Macrophages          1817
Neutrophils          1357
NK_cells              973
T_cells               300
Name: count, dtype: int64


Processing files: 100%|██████████| 5154/5154 [00:02<00:00, 2553.34it/s]

cluster
Fibroblasts          1351
Epithelial_cells     1247
Undefined             929
B_cells               644
Endothelial_cells     402
Macrophages           267
Neutrophils           152
NK_cells              103
T_cells                59
Name: count, dtype: int64
